# Preparation

## Clone the Repo

In [1]:
!git clone https://github.com/nianlonggu/MemSum.git

Cloning into 'MemSum'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 275 (delta 7), reused 2 (delta 2), pack-reused 266
Receiving objects: 100% (275/275), 82.32 MiB | 12.27 MiB/s, done.
Resolving deltas: 100% (104/104), done.


## Change the working directory to the main folder of MemSum

In [2]:
import os
os.chdir("MemSum")
!ls

config
data
Data_processing_training_and_testing_for_MemSum.ipynb
human_eval_results
images
MemSum_Human_Evaluation.ipynb
oracle_score_compute.ipynb
README.md
requirements.txt
slides_for_paper.pdf
src
summarizers.py


## Install packages

Note: Because colab has preinstalled torch, so we don't need to install pytorch again

We tested on torch version>=1.11.0.

In [3]:
!pip install -r requirements.txt -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
import torch
torch.__version__

'2.0.0+cu118'

# Preprocessing Custom data

Suppose that you have already splitted the training / validation and  test set:

The training data is now stored in a .jsonl file that contains a list of json info, one line for one training instance. Each json (or dictonary) contains two keys: 

1. "text": the value for which is a python list of sentences, this represents the document you want to summarize;
2. "summary": the value is also a list of sentences. If represent the ground-truth summary. Because the summary can contain multiple sentences, so we store them as a list.

The same for the validation file and the testing file. 


In [5]:
import json
# train_corpus = [ json.loads(line) for line in open("data/custom_data/train_CUSTOM_raw.jsonl") ]
import pandas as pd

df = pd.read_csv('data/corpusfinal.csv')
train_corpus = []
for i in range(len(df)):
  dick = {}
  dick['text'] = df['document'][i]
  dick['summary'] = df['summary'][i]
  train_corpus.append(dick)

## as an example, we have 100 instances for training
print(len(train_corpus))
print(train_corpus[0].keys())
print(train_corpus[0]["text"][:3])
print(train_corpus[0]["summary"][:3])

1977
dict_keys(['text', 'summary'])
The
The


If you have your own data, process them into the same structure then put them into the data/ folder

The next thing we need to do is to create high-ROUGE episodes for the training set, as introduced in the paper: https://aclanthology.org/2022.acl-long.450/,
and the github introduction: https://github.com/nianlonggu/MemSum#addition-info-code-for-obtaining-the-greedy-summary-of-a-document-and-creating-high-rouge-episodes-for-training-the-model

In [6]:
from src.data_preprocessing.MemSum.utils import greedy_extract
import json
from tqdm import tqdm

In [14]:
df = pd.read_csv('data/corpusfinal.csv')
df_train = df.iloc[:int(len(df)*0.85)]
train_corpus = []
for i in range(len(df_train)):
  dick = {}
  dick['text'] = df_train['document'][i].split(".")
  dick['summary'] = df_train['summary'][i].split(".")
  train_corpus.append(dick)

# train_corpus[0]
for data in tqdm(train_corpus):
    high_rouge_episodes = greedy_extract( data["text"], data["summary"], beamsearch_size = 2 )
    indices_list = []
    score_list  = []

    for indices, score in high_rouge_episodes:
        indices_list.append( indices )
        score_list.append(score)

    data["indices"] = indices_list
    data["score"] = score_list

100%|██████████| 1680/1680 [02:42<00:00, 10.34it/s]


Now we have obtained the labels for the training set. This can be parallized if you have large training set.

We can save the labeled training set to a new file:

In [15]:
with open("data/train_CUSTOM_labelled.jsonl","w") as f:
    for data in train_corpus:
        f.write(json.dumps(data) + "\n")

That's it! We are about to train MemSum!

In [16]:
df_test = df.iloc[int(len(df)*0.85):]
test_corpus = []
df_test
for i,row in df_test.iterrows():
  dick1 = {}
  dick1['text'] = row['document'].split(".")
  dick1['summary'] = str(row ['summary']).split(".")
  test_corpus.append(dick)
with open("data/val_CUSTOM_raw.jsonl","w") as f:
    for data in test_corpus:
        f.write(json.dumps(data) + "\n")

# Training

## Download pretrained word embedding

MemSUM used the glove embedding (200dim), with three addition token embeddings for bos eos pad, etc.

You can download the word embedding (a folder named glove/) used in this work:

https://drive.google.com/drive/folders/1lrwYrrM3h0-9fwWCOmpRkydvmF6hmvmW?usp=sharing

and put the folder under the model/ folder. 

Or you can do it using the code below:

Make sure the structure looks like:

1. MemSum/model/glove/unigram_embeddings_200dim.pkl
2. MemSum/model/glove/vocabulary_200dim.pkl


If not, you can change manually





In [17]:
!pip install gdown -q
try:
    os.system("rm -r model")
    os.makedirs("model/")
except:
    pass
!cd model/; gdown --folder https://drive.google.com/drive/folders/1lrwYrrM3h0-9fwWCOmpRkydvmF6hmvmW


if not os.path.exists("model/glove"):
    try:
        os.makedirs("model/glove")
        os.system("mv model/*.pkl model/glove/")
    except:
        pass

Retrieving folder list
Processing file 1SVTHcgWJDvoVCsLfdvkaw5ICkihjUoaH unigram_embeddings_200dim.pkl
Processing file 1SuF4HSe0-IBKWGtc1xqlzMHNDneiLi4- vocabulary_200dim.pkl
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1SVTHcgWJDvoVCsLfdvkaw5ICkihjUoaH
To: /content/MemSum/model/glove/unigram_embeddings_200dim.pkl
100% 320M/320M [00:05<00:00, 58.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SuF4HSe0-IBKWGtc1xqlzMHNDneiLi4-
To: /content/MemSum/model/glove/vocabulary_200dim.pkl
100% 4.16M/4.16M [00:00<00:00, 179MB/s]
Download completed


## Start training

Note:
1. you need to switch to the folder src/MemSum_Full;
2. You can specify the path to training and validation set, the model_folder (where you want to store model checkpoints) and the log_folder (where you want to store the log info), and other parameters. 
3. You can provide the absolute path, or relative path, as shown in the example code below.
4. n_device means the number of available GPUs

In [18]:
!cd src/MemSum_Full; python train.py -training_corpus_file_name ../../data/train_CUSTOM_labelled.jsonl -validation_corpus_file_name ../../data/val_CUSTOM_raw.jsonl -model_folder ../../model/MemSum_Full/custom_data/200dim/run1/ -log_folder ../../log/MemSum_Full/custom_data/200dim/run1/ -vocabulary_file_name ../../model/glove/vocabulary_200dim.pkl -pretrained_unigram_embeddings_file_name ../../model/glove/unigram_embeddings_200dim.pkl -max_seq_len 100 -max_doc_len 500 -num_of_epochs 5 -save_every 1000 -n_device 1 -batch_size_per_device 2 -max_extracted_sentences_per_document 7 -moving_average_decay 0.999 -p_stop_thres 0.6

1680it [00:00, 65670.98it/s]
297it [00:00, 55646.76it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
0it [00:00, ?it/s]/content/MemSum/src/MemSum_Full/train.py:227: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  remaining_mask_np = np.ones_like(

# Testing trained model on custom dataset

In [19]:
from summarizers import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np

In [26]:
rouge_cal = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeLsum'], use_stemmer=True)

memsum_custom_data = MemSum( "model/MemSum_Full/custom_data/200dim/run1/model_batch_3360.pt", 
                  "model/glove/vocabulary_200dim.pkl", 
                  gpu = 0 ,  max_doc_len = 500  )

In [22]:
df_val = pd.read_csv('data/corpusfinal.csv')
df_test = df.iloc[:int(len(df)*0.3)]
test_corpus_custom_data = []
df_test
for i,row in df_test.iterrows():
  dick1 = {}
  dick1['text'] = row['document'].split(".")
  dick1['summary'] = str(row['summary']).split(".")
  test_corpus_custom_data.append(dick)

In [23]:
def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )
    
    return np.asarray(scores).mean(axis = 0)

In [28]:
evaluate( memsum_custom_data, test_corpus_custom_data, 0.6, 7, rouge_cal )

  0%|          | 1/593 [00:00<02:00,  4.93it/s]

['Sampling of 1367 trees was conducted in the Side wildfire (4 May 1996), Bridger-Knoll wildfire (20 June 1996) and Dauber prescribed fire (9 September 1995) in northern Arizona ponderosa pine forests (Pinus ponderosa)', ' Tree mortality was assessed for 3 years after each fire', ' Compared with trees that lived for 3 years after fire, dead trees in all fires had more crown scorch, crown consumption, bole scorch, ground char, and bark beetle attacks', ' Logistic regression models were used to provide insight on factors associated with tree mortality after fire', ' The amount of total crown damage associated with the onset of tree mortality decreased as bole char severity increased', ' Models using diameter at breast height (dbh) and crown volume damage suggested that tree mortality decreased as dbh increased in the Dauber prescribed fire where trees were smallest, and tree mortality increased as dbh increased in the Side and Bridger-Knoll wildfires where trees were largest', '']
['Samp

  1%|          | 3/593 [00:00<01:39,  5.96it/s]

['Sampling of 1367 trees was conducted in the Side wildfire (4 May 1996), Bridger-Knoll wildfire (20 June 1996) and Dauber prescribed fire (9 September 1995) in northern Arizona ponderosa pine forests (Pinus ponderosa)', ' Logistic regression models were used to provide insight on factors associated with tree mortality after fire', ' Tree mortality was assessed for 3 years after each fire', ' Compared with trees that lived for 3 years after fire, dead trees in all fires had more crown scorch, crown consumption, bole scorch, ground char, and bark beetle attacks', ' In the Dauber and Side fires, 95% and 94% of 3-year post-fire mortality occurred by year 2, versus 76% in the Bridger-Knoll wildfire', ' The amount of total crown damage associated with the onset of tree mortality decreased as bole char severity increased', '9% in the Bridger-Knoll wildfire']
['Sampling of 1367 trees was conducted in the Side wildfire (4 May 1996), Bridger-Knoll wildfire (20 June 1996) and Dauber prescribed f

  1%|          | 5/593 [00:00<01:35,  6.18it/s]

...

100%|█████████▉| 591/593 [01:37<00:00,  6.18it/s]

['Sampling of 1367 trees was conducted in the Side wildfire (4 May 1996), Bridger-Knoll wildfire (20 June 1996) and Dauber prescribed fire (9 September 1995) in northern Arizona ponderosa pine forests (Pinus ponderosa)', ' Logistic regression models were used to provide insight on factors associated with tree mortality after fire', ' Tree mortality was assessed for 3 years after each fire', ' Compared with trees that lived for 3 years after fire, dead trees in all fires had more crown scorch, crown consumption, bole scorch, ground char, and bark beetle attacks', ' In the Dauber and Side fires, 95% and 94% of 3-year post-fire mortality occurred by year 2, versus 76% in the Bridger-Knoll wildfire', ' The amount of total crown damage associated with the onset of tree mortality decreased as bole char severity increased', '9% in the Bridger-Knoll wildfire']
['Sampling of 1367 trees was conducted in the Side wildfire (4 May 1996), Bridger-Knoll wildfire (20 June 1996) and Dauber prescribed f

100%|██████████| 593/593 [01:37<00:00,  6.07it/s]

['Sampling of 1367 trees was conducted in the Side wildfire (4 May 1996), Bridger-Knoll wildfire (20 June 1996) and Dauber prescribed fire (9 September 1995) in northern Arizona ponderosa pine forests (Pinus ponderosa)', ' Logistic regression models were used to provide insight on factors associated with tree mortality after fire', ' Tree mortality was assessed for 3 years after each fire', ' Compared with trees that lived for 3 years after fire, dead trees in all fires had more crown scorch, crown consumption, bole scorch, ground char, and bark beetle attacks', ' In the Dauber and Side fires, 95% and 94% of 3-year post-fire mortality occurred by year 2, versus 76% in the Bridger-Knoll wildfire', ' The amount of total crown damage associated with the onset of tree mortality decreased as bole char severity increased', '9% in the Bridger-Knoll wildfire']
['Sampling of 1367 trees was conducted in the Side wildfire (4 May 1996), Bridger-Knoll wildfire (20 June 1996) and Dauber prescribed f

array([0.82312925, 0.74657534, 0.80952381])

In [ ]:
!zip -r ../memsummodel.zip /content/MemSum/model

To cite MemSum, please use the following bibtex:

```
@inproceedings{gu-etal-2022-memsum,
    title = "{M}em{S}um: Extractive Summarization of Long Documents Using Multi-Step Episodic {M}arkov Decision Processes",
    author = "Gu, Nianlong  and
      Ash, Elliott  and
      Hahnloser, Richard",
    booktitle = "Proceedings of the 60th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers)",
    month = may,
    year = "2022",
    address = "Dublin, Ireland",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2022.acl-long.450",
    doi = "10.18653/v1/2022.acl-long.450",
    pages = "6507--6522",
    abstract = "We introduce MemSum (Multi-step Episodic Markov decision process extractive SUMmarizer), a reinforcement-learning-based extractive summarizer enriched at each step with information on the current extraction history. When MemSum iteratively selects sentences into the summary, it considers a broad information set that would intuitively also be used by humans in this task: 1) the text content of the sentence, 2) the global text context of the rest of the document, and 3) the extraction history consisting of the set of sentences that have already been extracted. With a lightweight architecture, MemSum obtains state-of-the-art test-set performance (ROUGE) in summarizing long documents taken from PubMed, arXiv, and GovReport. Ablation studies demonstrate the importance of local, global, and history information. A human evaluation confirms the high quality and low redundancy of the generated summaries, stemming from MemSum{'}s awareness of extraction history.",
}
```